In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import random
import csv
import json

In [4]:
# 1 - Login to Linkedin

# Open webdriver and login
driver = webdriver.Chrome()
url = 'https://www.linkedin.com/login'
driver.get(url)
sleep(random.uniform(2, 3))

credential = open('Login_credentials.txt')
line = credential.readlines()
username = line[0]
password = line[1]

email_field = driver.find_element_by_id('username')
email_field.send_keys(username)
sleep(random.uniform(2, 3))

password_field = driver.find_element_by_name('session_password')
password_field.send_keys(password)
sleep(random.uniform(2, 3))

login_field = driver.find_element_by_xpath('//*[@id="organic-div"]/form/div[3]/button')
login_field.click()
sleep(random.uniform(2, 3))


In [14]:
# Go to connection list

driver.get('https://www.linkedin.com/mynetwork/invite-connect/connections/')
sleep(random.uniform(1, 3))


In [ ]:
# 2 - Extract profiles from the page

page_source = BeautifulSoup(driver.page_source, 'html.parser')
profiles = page_source.find('section', {'class' : 'mn-connections mb4 artdeco-card ember-view'})
profiles_ul = profiles.find('ul')
# profiles_li = profiles_ul.find_all('li')
scroll_up = driver.find_element_by_tag_name('body')

all_profiles_URL = []
while len(all_profiles_URL) < 100:
    for profile in profiles_ul.find_all('li'):
        scroll_up.send_keys(Keys.ARROW_UP)
        sleep(1)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        profile_ID = profile.find('a', {'class' : 'ember-view mn-connection-card__picture'})
        profile_URL = "https://www.linkedin.com" + profile_ID.get('href')
        if profile_URL not in all_profiles_URL:
            all_profiles_URL.append(profile_URL)
        sleep(2)

print('- Finish extract -')
    

In [ ]:
# test

page_source = BeautifulSoup(driver.page_source, 'html.parser')
profiles = page_source.find('section', {'class' : 'mn-connections mb4 artdeco-card ember-view'})
profiles_ul = profiles.find('ul')
# profiles_li = profiles_ul.find_all('li')
scroll_up = driver.find_element_by_tag_name('body')

for i in range(50):
    scroll_up.send_keys(Keys.ARROW_UP)
    sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    sleep(2)
    
all_profiles_URL = []
for profile in profiles_ul.find_all('li'):
    profile_ID = profile.find('a', {'class' : 'ember-view mn-connection-card__picture'})
    profile_URL = "https://www.linkedin.com" + profile_ID.get('href')
    if profile_URL not in all_profiles_URL:
        all_profiles_URL.append(profile_URL)

print('- Finish extract -')


In [ ]:
# 3 - Scrape the data of 1 Linkedin profile, and write the data to a .CSV file

with open('output.csv', 'w',  newline = '', encoding='utf-8-sig') as file_output:
    headers = ['Name', 'Job Title', 'Location', 'Email', 'Phone', 'URL']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    
    for linkedin_URL in all_profiles_URL:
        driver.get(linkedin_URL)
        # get job title
        page_source = BeautifulSoup(driver.page_source, 'html.parser')
        title = page_source.find('div', {'class' : 'text-body-medium break-words'}).get_text().strip()
        sleep(3)
        driver.get(linkedin_URL + 'detail/contact-info/')
        print('- Accessing profile: ', linkedin_URL)

        page_source = BeautifulSoup(driver.page_source, 'html.parser')
        # get name
        info_name = page_source.find('section', {'class' : 'pv-contact-info__contact-type ci-vanity-url'}).get_text().strip()
        name = info_name.split('’')[0]
#         print('--- Profile name is: ', name)
        # get URL
        info_URL = page_source.find('div', {'class' : 'pv-contact-info__ci-container t-14'})
        URL = info_URL.find('a').get('href')
        # get phone
        info_phone = page_source.find('section', {'class' : 'pv-contact-info__contact-type ci-phone'})
        try:
            info_phone_li = info_phone.find('li').get_text().strip().split('\n')
            phone = '\'' + info_phone_li[0]
        except AttributeError:
            phone = 'None'
#         print('--- Profile phone is: ', phone)
        # get address
        info_address = page_source.find('section', {'class' : 'pv-contact-info__contact-type ci-address'})
        try:
            info_address_div = info_address.find('div')
            address = info_address_div.find('a').get_text().strip()
        except AttributeError:
            address = 'None'
#         print('--- Profile address is: ', address)
        # get email
        info_email = page_source.find('section', {'class' : 'pv-contact-info__contact-type ci-email'})
        info_email_div = info_email.find('div')
        email = info_email_div.find('a').get_text().strip()
#         print('--- Profile email is: ', email)

        writer.writerow({headers[0]:name, headers[1]:title, headers[2]:address, headers[3]:email, headers[4]:phone, headers[5]:URL})
        print('\n')
        sleep(5)

print('- Mission Completed! -')